
# Week06 — LoRA 기반 SFT 실습 (Local 환경용)

**목표**
- 작은 예제 데이터로 **LoRA 기반 SFT**를 로컬에서 수행해보고,
- **Base vs LoRA** 추론 결과를 비교/기록합니다.
- (선택) 로컬 **Ollama `llama3.1:8b-instruct`**와 결과를 간단 비교합니다.

> ⚙️ 본 노트북은 **로컬 실행**을 전제로 하며, GPU가 있으면 `4/8bit` 로딩을 활용합니다. CPU 환경에서도 **작은 모델**로 데모는 가능합니다.


## 0) 환경 준비

In [1]:

# !pip install -q -U transformers datasets peft accelerate bitsandbytes sentencepiece#   langfuse python-dotenv pandas requests torch --extra-index-url https://download.pytorch.org/whl/cu121
# ↑ 필요 시 주석 해제. CUDA 버전에 맞춰 torch 인덱스는 조정하세요.
import sys, platform, torch, os, json, time, math, random
from pathlib import Path
print('Python:', sys.version)
print('Platform:', platform.platform())
print('CUDA available:', torch.cuda.is_available())
print('Torch:', torch.__version__)


Python: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
Platform: macOS-15.6.1-arm64-arm-64bit
CUDA available: False
Torch: 2.8.0


## 1) .env 로드 (OpenAI/Langfuse/Pinecone 키 등)

In [2]:

from dotenv import load_dotenv, find_dotenv
from pathlib import Path

# 프로젝트 루트에서 .env 탐색
env_path = find_dotenv(usecwd=True)
if not env_path:
    # 상위 디렉토리 탐색(노트북이 /week06 안에 있을 수 있으므로)
    cur = Path.cwd()
    for p in [cur] + list(cur.parents):
        cand = p / '.env'
        if cand.exists():
            env_path = str(cand)
            break

if env_path:
    load_dotenv(env_path)
    print(f"Loaded .env from: {env_path}")
else:
    print("⚠️ .env를 찾지 못했습니다. 프로젝트 루트에 .env를 생성/배치하세요.")

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')
LANGFUSE_PUBLIC_KEY = os.getenv('LANGFUSE_PUBLIC_KEY', '')
LANGFUSE_SECRET_KEY = os.getenv('LANGFUSE_SECRET_KEY', '')
LANGFUSE_HOST = os.getenv('LANGFUSE_HOST', 'https://cloud.langfuse.com')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', '')

print('OpenAI key set:', bool(OPENAI_API_KEY))
print('Langfuse host:', LANGFUSE_HOST)
print('Pinecone key set:', bool(PINECONE_API_KEY))


Loaded .env from: /Users/sjcha/Documents/3. 아주대AI대학원/2025-2nd-semester/ajou-llmops-2025-2nd-semester/.env
OpenAI key set: True
Langfuse host: https://cloud.langfuse.com
Pinecone key set: True


## 2) 실험 설정

In [8]:

from dataclasses import dataclass
from typing import Optional

@dataclass
class CFG:
    # 실행/경로
    project_root: Path = Path.cwd().resolve()
    data_dir: Path = Path('data')
    out_dir: Path = Path('runs/lora_sft')
    # 모델/토크나이저 (작은 모델 권장)
    base_model: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # CPU/GPU 모두 가능한 예시
    use_qlora: bool = True  # GPU 있으면 QLoRA, 아니면 자동 CPU
    # 토큰/길이
    max_seq_len: int = 512
    # LoRA 하이퍼파라미터
    lora_r: int = 8
    lora_alpha: int = 16
    lora_dropout: float = 0.05
    target_modules: tuple = ("q_proj","v_proj")
    # 학습 하이퍼
    lr: float = 2e-4
    warmup_ratio: float = 0.05
    num_epochs: int = 1
    per_device_train_batch_size: int = 2
    grad_accum_steps: int = 8
    logging_steps: int = 10
    save_steps: int = 50
    eval_ratio: float = 0.1  # 간이 검증셋 비율
    seed: int = 42

cfg = CFG()
cfg.data_dir.mkdir(parents=True, exist_ok=True)
cfg.out_dir.mkdir(parents=True, exist_ok=True)
cfg


CFG(project_root=PosixPath('/Users/sjcha/Documents/3. 아주대AI대학원/2025-2nd-semester/ajou-llmops-2025-2nd-semester/week06'), data_dir=PosixPath('data'), out_dir=PosixPath('runs/lora_sft'), base_model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', use_qlora=True, max_seq_len=512, lora_r=8, lora_alpha=16, lora_dropout=0.05, target_modules=('q_proj', 'v_proj'), lr=0.0002, warmup_ratio=0.05, num_epochs=1, per_device_train_batch_size=2, grad_accum_steps=8, logging_steps=10, save_steps=50, eval_ratio=0.1, seed=42)

## 3) 데이터 준비 (알파카 스타일 JSONL, 없으면 미니 예제 생성)

In [9]:

import json, random
from pathlib import Path

train_path = cfg.data_dir / 'train.jsonl'
val_path   = cfg.data_dir / 'val.jsonl'

def write_demo_jsonl(path: Path, n=60):
    demo = []
    for i in range(n):
        demo.append({
            "instruction": "요청한 주제에 대해 한 문단으로 간결히 설명하세요.",
            "input": f"주제: 데이터 품질과 라벨 마스킹의 중요성 #{i}",
            "output": "데이터 품질은 모델 성능의 상한을 결정하며, 라벨 마스킹 오류는 수렴을 방해한다. 일관된 템플릿과 PII/Toxic 필터는 필수다."
        })
    with path.open('w', encoding='utf-8') as f:
        for r in demo:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

# 파일이 없으면 데모 생성
if not train_path.exists():
    write_demo_jsonl(train_path, n=80)
if not val_path.exists():
    write_demo_jsonl(val_path, n=10)

# 간단 확인
print('train lines:', sum(1 for _ in open(train_path, 'r', encoding='utf-8')))
print('val   lines:', sum(1 for _ in open(val_path,   'r', encoding='utf-8')))


train lines: 80
val   lines: 10


## 4) 토크나이저/데이터셋 로드 & 템플릿 적용

In [10]:

from datasets import load_dataset, Dataset

def read_jsonl(p: Path):
    rows = []
    with p.open('r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                rows.append(json.loads(line))
    return Dataset.from_list(rows)

train_ds = read_jsonl(train_path)
val_ds = read_jsonl(val_path)

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(cfg.base_model, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

INSTR_TMPL = "### Instruction\n{instruction}\n\n### Input\n{input}\n\n### Response\n"

def format_example(ex):
    prompt = INSTR_TMPL.format(instruction=ex.get("instruction","").strip(),
                               input=ex.get("input","").strip())
    target = ex.get("output","").strip()
    text = prompt + target
    enc = tokenizer(text, max_length=cfg.max_seq_len, truncation=True, padding='max_length')
    # 라벨 마스킹: prompt 부분은 -100으로
    prompt_ids = tokenizer(prompt, max_length=cfg.max_seq_len, truncation=True, padding='max_length')['input_ids']
    labels = [-100]*len(prompt_ids)
    full_ids = enc['input_ids']
    # prompt 길이 이후는 정답 라벨
    for i in range(len(full_ids)):
        if i >= len(prompt_ids) or prompt_ids[i] != full_ids[i]:
            labels[i] = full_ids[i]
    enc['labels'] = labels
    return enc

train_tok = train_ds.map(format_example, remove_columns=train_ds.column_names)
val_tok = val_ds.map(format_example, remove_columns=val_ds.column_names)
train_tok, val_tok


Map: 100%|██████████| 10/10 [00:00<00:00, 1865.38 examples/s]


(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 80
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 10
 }))

## 5) 모델 로드(4/8bit 옵션) & LoRA 적용

In [ ]:

from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

device_map = 'auto'
use_cuda = torch.cuda.is_available()
load_in_8bit = False
load_in_4bit = False
bnb_config = None

if use_cuda and cfg.use_qlora:
    try:
        from bitsandbytes.config import BitsAndBytesConfig
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
        )
        load_in_4bit = True
        print("Using 4bit QLoRA.")
    except Exception as e:
        print("4bit 불가 → 8bit 시도:", e)
        load_in_8bit = True

model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    # CPU/MPS에서는 device_map=None + float32 로 깔끔히 로딩
    device_map=None,
    low_cpu_mem_usage=False,          # ← meta 텐서 경로 회피
    dtype=torch.float32,
    load_in_8bit=load_in_8bit,
    quantization_config=bnb_config if load_in_4bit else None,
).to("cuda" if use_cuda else "cpu")


if (load_in_4bit or load_in_8bit) and hasattr(model, "enable_input_require_grads"):
    model = prepare_model_for_kbit_training(model)

lora_cfg = LoraConfig(
    r=cfg.lora_r,
    lora_alpha=cfg.lora_alpha,
    lora_dropout=cfg.lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=list(cfg.target_modules),
)
model = get_peft_model(model, lora_cfg)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / Total: {total:,} ({100*trainable/total:.2f}%)")


`torch_dtype` is deprecated! Use `dtype` instead!


Trainable params: 1,126,400 / Total: 1,101,174,784 (0.10%)


## 6) 학습 실행 (Trainer)

In [ ]:

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import math, os

args = TrainingArguments(
    output_dir=str(cfg.out_dir),
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    gradient_accumulation_steps=cfg.grad_accum_steps,
    learning_rate=cfg.lr,
    num_train_epochs=cfg.num_epochs,
    fp16=torch.cuda.is_available(),
    bf16=torch.cuda.is_available(),
    logging_steps=cfg.logging_steps,
    save_steps=cfg.save_steps,
    optim="paged_adamw_8bit" if (load_in_4bit or load_in_8bit) else "adamw_torch",
    warmup_ratio=cfg.warmup_ratio,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok.select(range(max(1, int(len(val_tok)*cfg.eval_ratio)))),
    data_collator=data_collator,
)

res = trainer.train()
trainer.save_model(cfg.out_dir / "adapter")  # LoRA 어댑터 저장
(tokenizer.save_pretrained(cfg.out_dir / "adapter"))

# 간단 로그
final_loss = res.training_loss if hasattr(res, 'training_loss') else None
print("Final training loss:", final_loss)


/Users/sjcha/Documents/3. 아주대AI대학원/2025-2nd-semester/ajou-llmops-2025-2nd-semester/ajou-llmops/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


## 7) 추론: Base vs LoRA 비교

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def build_prompt(instruction:str, _input:str):
    return INSTR_TMPL.format(instruction=instruction.strip(), input=_input.strip())

# 7-1) Base 모델 (어댑터 미적용) 로드
base_tok = tokenizer  # 같은 토크나이저 사용
base_model = AutoModelForCausalLM.from_pretrained(
    cfg.base_model,
    device_map=device_map,
    load_in_8bit=load_in_8bit,
    quantization_config=bnb_config if load_in_4bit else None,
)

# 7-2) LoRA 적용 모델은 trainer.model 재사용
gen_kwargs = dict(max_new_tokens=200, do_sample=False, temperature=0.0, top_p=1.0)

tests = [
    ("회의록 요약", "아래 회의 메모를 3줄로 요약하세요. 메모: 데이터 검수, 템플릿 고정, β 스윕 계획"),
    ("데이터 품질 체크리스트", "SFT 데이터셋을 제출하기 전 수행해야 할 체크 5가지를 불릿으로."),
]

def generate(m, tok, prompt):
    ids = tok(prompt, return_tensors='pt').to(m.device)
    with torch.no_grad():
        out = m.generate(**ids, **gen_kwargs)
    return tok.decode(out[0], skip_special_tokens=True).split("### Response")[-1].strip()

for name, content in tests:
    prompt = build_prompt(name, content)
    base_out = generate(base_model, base_tok, prompt)
    lora_out = generate(model, tokenizer, prompt)
    print("\n" + "="*80)
    print("PROMPT:", name)
    print("- Base:"); print(base_out[:1000])
    print("- LoRA:"); print(lora_out[:1000])


## (선택) 8) Ollama `llama3.1:8b-instruct`와 간단 비교

In [ ]:

import requests, json, os

OLLAMA_HOST = os.getenv('OLLAMA_HOST', 'http://localhost:11434')
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'llama3.1:8b-instruct')

def ollama_generate(prompt, model=OLLAMA_MODEL, host=OLLAMA_HOST):
    try:
        resp = requests.post(f"{host}/api/generate", json={"model": model, "prompt": prompt, "stream": False}, timeout=60)
        resp.raise_for_status()
        return resp.json().get("response","").strip()
    except Exception as e:
        print("Ollama 요청 실패:", e)
        return None

if __name__ == "__main__":
    p = build_prompt("데이터 검증 규칙", "JSONL 스키마와 라벨 마스킹 체크 항목을 5개로 요약.")
    print("Ollama 요청...")
    o = ollama_generate(p)
    if o:
        print(o)
    else:
        print("⚠️ Ollama 서버가 실행 중인지 확인하세요 (default: localhost:11434).")


## 9) 결과 기록 저장 (CSV)

In [ ]:

import pandas as pd, time

records = []
for name, content in tests:
    prompt = build_prompt(name, content)
    base_out = generate(base_model, base_tok, prompt)
    lora_out = generate(model, tokenizer, prompt)
    records.append({
        "prompt_name": name,
        "prompt_input": content,
        "base_out": base_out,
        "lora_out": lora_out,
        "base_len": len(base_out),
        "lora_len": len(lora_out),
    })

df = pd.DataFrame(records)
out_csv = cfg.out_dir / f"compare_{int(time.time())}.csv"
df.to_csv(out_csv, index=False, encoding='utf-8')
print("Saved:", out_csv)
df.head()


## (선택) 10) Langfuse 간단 로깅

In [ ]:

if LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY:
    try:
        from langfuse import Langfuse
        lf = Langfuse(public_key=LANGFUSE_PUBLIC_KEY, secret_key=LANGFUSE_SECRET_KEY, host=LANGFUSE_HOST)
        obs = lf.trace(name="week06_lora_sft")
        obs.generation(name="config", metadata={"cfg": {**cfg.__dict__}})
        for r in records:
            obs.event(name="compare", input=r["prompt_input"], output=r["lora_out"], metadata={"base_len": r["base_len"], "lora_len": r["lora_len"]})
        obs.end()
        print("Logged to Langfuse.")
    except Exception as e:
        print("Langfuse 로깅 실패:", e)
else:
    print("Langfuse 키가 없어 로깅 생략.")



## 11) 로컬 실행 팁

- **GPU 권장**: QLoRA(4bit) 사용 시 8–12GB VRAM에서도 1B 모델 학습이 가능합니다.  
- **CPU만 있을 때**: `cfg.use_qlora=False`로 두고, 매우 작은 모델(예: 500M〜1B)을 사용하세요. 속도는 느리지만 개념 체득은 가능합니다.
- **메모리 부족(OOM)**: `max_seq_len↓`, `per_device_train_batch_size↓`, `grad_accum_steps↑`, 또는 8/4bit 로딩을 활용하세요.
- **템플릿 일관성**: 학습/평가/서빙에서 동일 템플릿(BOS/EOS/SEP/구분자)을 사용하세요.
- **데이터 품질**: SFT의 성패는 데이터에 달려 있습니다. JSONL 스키마/중복/금지어를 꼭 점검하세요.
